# Analysis

This document implements an initial evaluation and analysis of the wind turbines dataset provided by Engie.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import windrose
import os

from sqlalchemy import create_engine
from dotenv import load_dotenv

## Wind Turbines: A Quick Primer

Before we even look at the available data, it is worth building a little domain knowledge in the function of wind turbines.

#### Basic Terminology and Operations

Wind turbines are mechanically complex, but the basic ideas are simple:

* The *nacelle* is the housing for all the main components of the wind turbine, including the gearbox, generator, drive train, and brake assembly. The temperature outside the nacelle is represented by the `ot` variable; the temperature inside the nacelle is represented by the `yt` variable. It can get quite hot inside the nacelle, given the amounts of energy that being transmitted!
* The nacelle is rotated by the yaw drive, which allows the turbine to face the wind. In our dataset, this nacelle angle is represented by the `ya` variable.
* The rotor is spun by the wind and is connected to the generator via the gearbox. The rotor speed is represented by the `rs` variable, while the rotor torque is represented by the `rm` variable.
* The rotor is supported by a main bearing, which is lubricated by oil. The temperature of the oil is represented by the `rbt` variable.
* Rotor speed can be controlled by adjusting the pitch angle of the blades. This is represented by the `ba` variable.
* Our wind turbines are the *Senvion MM82* model. This model starts working at a wind speed of 3.5 m/s and has a cut-out speed of 25 m/s. It has a rated wind speed of 14.5 m/s and a rated power of 2,050kW. This is equivalent to about one-four-hundredth of the power of a nuclear reactor. It also has a maximum rotor speed of 17 rpm.
* The rotor is connected to a gearbox, which increases the speed of the rotor to the speed of the generator, while simultaneously stepping down the torque.
* Looking up the model details, we find that the generator is an asynchronous (induction) generator, which means that the rotor speed is not directly related to the frequency of the electrical current that is generated. It produces 690V AC at 50Hz.

We also need to know a little bit about electrical engineering and power generation:

* Active power (i.e. "real power") is the power that is actually produced by a generator. This is represented by the `p` variable, and is measured in kW.
* Reactive power is the power that is not generated, but is instead used "to maintain the electric and magnetic fields of the generator." This is represented by the `q` variable, and is measured in kVAr.
* So, we can say something like: "Active power is the power generated by the wind turbine that does actual work on the load, while reactive power is the power associated with the energy exchange in the electrical system's inductive and capacitive components, needed to maintain voltage levels and grid stability, thus ensuring the quality and efficient delivery of active power." The relationship between these two is not so simple; here are three examples that ChatGPT gave me:

    1. High wind speeds with high grid demand: In this scenario, the wind turbine generates a large amount of active power due to strong wind conditions. However, since the grid demand is also high, the turbine's control system must manage reactive power to maintain grid stability. In this case, both active and reactive power outputs can be high, but the reactive power may still be significantly different from the active power to meet the grid's voltage and stability requirements.

    2. Moderate wind speeds with low grid demand: In this situation, the wind turbine generates a moderate amount of active power due to average wind speeds. However, since the grid demand is low, there might be less need for reactive power support. As a result, the turbine's control system can focus on maximizing active power output while minimizing reactive power production. In this case, active power will be relatively high compared to reactive power.

    3. Unsteady wind speeds with varying grid demand: In this scenario, wind speeds fluctuate frequently, causing the active power output of the wind turbine to vary accordingly. Meanwhile, the grid demand also changes, requiring the turbine to adjust its reactive power support. In this case, the relationship between active and reactive power can be highly dynamic, with significant differences observed at different times depending on the specific wind speed and grid demand conditions.

* Apparent power is the vector sum of active and reactive power, and the power factor is then the ratio of active power to apparent power.
* The power factor is therefore a measure of the efficiency of the generator. A power factor of 1 means that all the power is being used to generate electricity, while a power factor of 0 means that all the power is being used to maintain the electric and magnetic fields of the generator.
* The power factor is directly determined by the excitation level of the generator, which is set by the generator's control systems.
* These control systems are complicated! They must take into account both the available kinetic energy from the rotor and the demands of the grid, all while ensuring that the operation of the turbine remains within safe limits.

Some other important facts:

* Rotor power on a wind turbine is proportional to the cube of the wind speed. This means that a small increase in wind speed can lead to a large increase in power generation.
* Rotor torque meanwhile is proportional to the square of the wind speed.
* The energy content of the wind is largely determined by the wind speed and the air density. Air density is in turn affected by the temperature, pressure and humidity.
* There is a theoretical maximum efficiency of 59.3% for wind turbines, known as the Betz limit. This is the maximum amount of energy that can be extracted from the wind, and is a function of the wind speed. In practice, wind turbines operate at around 40% efficiency.
* The tip speed ratio is the ratio of the speed of the blade tips to the speed of the wind. This is a function of the rotor speed and the wind speed. The tip speed ratio is important because it determines the efficiency of the turbine. The optimal tip speed ratio is around 6, and the turbine will automatically adjust the rotor speed to maintain this ratio.
* Per (Hau, 2005):

> Exceeding the maximum permissible service temperature of the lubricating oil or the bearings of the rotor shaft, of the gearbox and of the electric generator has been found to be a frequent source of trouble in many wind turbines.

#### The Gist of It

The objective of a wind turbine is to produce the maximum possible amount of power, while remaining within the operating limits of the turbine.

However, this does not mean simply spinning the rotor as fast as possible! This is because each rotor has an optimal speed for any given wind speed (this ratio is referred to as the *tip speed ratio*).

For any given rotor speed, we will also observe a rotor torque. This is obviously a function of rotor speed, but it is also a function of the available energy in the wind, which is in turn a function of the wind speed and the air density.

So the turbine's control systems will ensure that the rotor is spun at the optimal speed for the given wind speed. The gearbox will then transform this rotor speed into a speed more suitable for the generator, and step down the torque.

The input to the generator is therefore a given rotor speed and torque. The control systems of the generator will then manage the excitation level of the generator, to ensure that the maximum amount of power is being generated whilst also maintaining the stability of the grid. These outputs are captured by the active power and reactive power variables.


For fun, here is a photo:

![the wind turbine](references/turbine-senvion_mm82-DP71t6EmCB2.jpg)

## Data Discovery

Now that we have a basic understanding of wind turbines and their function, we can start to build an understanding of the available data. This means studying its 'shape' (i.e. dimensionality) and 'texture' (i.e. quality, distribution).

### 10,000 ft view

The primary dataset is composed of hourly observations of 14 mechanical metrics taken from four wind turbines located across the Grand Est region in France, paired with hourly weather observations across 7 dimensions. This means a total of 21 numeric variables, indexed by hour and turbine.

The data is available in a local Postgres database, which we can connect to using the `sqlalchemy` library.

In [2]:
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

engine = create_engine(DATABASE_URL)
conn = engine.connect()

There are just over one million observations, with roughly 261,000 observations for each of the four turbines, dated between January 2013 and January 2018. In fact, the four turbines have exactly the same date range, but each are missing some records across the period.

In [3]:
results = pd.read_sql("""
SELECT
    wind_turbine_name,
    min(datetime) AS min_datetime,
    max(datetime) AS max_datetime,
    count(*) AS count
FROM
    wind_turbines.observations
GROUP BY
    wind_turbine_name
""", conn)
results.head()

,wind_turbine_name,min_datetime,max_datetime,count
0,R80711,2012-12-31 23:00:00+00:00,2018-01-12 23:00:00+00:00,261812
1,R80721,2012-12-31 23:00:00+00:00,2018-01-12 23:00:00+00:00,261471
2,R80736,2012-12-31 23:00:00+00:00,2018-01-12 23:00:00+00:00,261138
3,R80790,2012-12-31 23:00:00+00:00,2018-01-12 23:00:00+00:00,261414


In [4]:
results = pd.read_sql("""
SELECT
    date_trunc('month', datetime) :: date AS month,
    SUM(
        CASE WHEN wind_turbine_name = 'R80711' THEN 1 ELSE 0 END
        ) AS R80711,
    SUM(
        CASE WHEN wind_turbine_name = 'R80721' THEN 1 ELSE 0 END
        ) AS R80721,
    SUM(
        CASE WHEN wind_turbine_name = 'R80736' THEN 1 ELSE 0 END
        ) AS R80736,
    SUM(
        CASE WHEN wind_turbine_name = 'R80790' THEN 1 ELSE 0 END
        ) AS R80790
FROM
    wind_turbines.observations
GROUP BY
    1
ORDER BY
    1
""", conn)
results.head(100)

,month,r80711,r80721,r80736,r80790
0,2013-01-01,4403,4403,4403,4403
1,2013-02-01,4032,4032,4032,4032
2,2013-03-01,4314,4314,4314,4314
3,2013-04-01,4312,4314,4314,4318
4,2013-05-01,4320,4320,4320,4320
...,...,...,...,...,...
56,2017-09-01,4046,4078,3964,4002
57,2017-10-01,4209,4363,4321,4215
58,2017-11-01,4046,4058,3954,4031
59,2017-12-01,4363,4338,4102,4361


For fun, let's plot the locations of the wind turbines on a map.

In [5]:
static_information = pd.read_sql("""
SELECT * FROM wind_turbines.static_information
""", conn)

coords = static_information[['wind_turbine_name', 'gps']].to_dict(orient='records')
for coord in coords:
    coord['gps'] = [float(x) for x in coord['gps'].split(',')]
    coord['lat'] = coord['gps'][0]
    coord['lon'] = coord['gps'][1]

mean_lat = sum([coord['lat'] for coord in coords]) / len(coords)
mean_lon = sum([coord['lon'] for coord in coords]) / len(coords)

m = folium.Map(location=[mean_lat, mean_lon], zoom_start=13)

for coord in coords:
    folium.Marker(
        location=[coord['lat'], coord['lon']],
        popup=coord['wind_turbine_name'],
        icon=folium.Icon(color='green', icon='ok-sign')
    ).add_to(m)

m

It looks like all four turbines are located right next to each other, suggesting they will likely share similar weather conditions (and perhaps similar power generation profiles).

As we continue to explore the data, it will be important to keep in mind potential issues:
* From the discrepencies in the time series, we can infer that the data are not totally continuous. This can cause issues in time series modelling and may need to be addressed later on, depending on the modelling approach chosen and the reasons for the gaps.
* Aside from gaps in the time series, there may be some missing values acrosss the observations. Most learning algorithms cannot handle missing values, so these would likely need to be imputed or the records dropped.
* Even where the values are not missing, they may be 'incorrect' in some sense. This is difficult to identify without significant domain expertise, but may become more clear when we study the distributions later.
* We have data across four different wind turbines and it may not be appropriate to treat these as equivalent ('exchangeable'). Whilst the turbines are indeed adjacent, they may still exhibit different mechanical properties and thus different power generation profiles. We may need to build separate models for each turbine, or at least include a turbine identifier as a feature in our models.

### Variables

The 14 mechanical metrics and 7 weather metrics can be distinguished into a few categories:

* Mechanical – Wind
    * `ws`: wind speed in m/s
    * `ws_1`: wind speed in m/s for the first anemometer on the nacelle
    * `ws_2`: wind speed in m/s for the second anemometer on the nacelle
    * `wa`: wind direction in degrees
* Mechanical – Positioning
    * `ba`: pitch angle of the blades in degrees
    * `va`: vane position, which is the angular position of the wind vane, in degrees
    * `ya`: nacelle angle in degrees
* Mechanical – Temperatures
    * `ot`: outdoor temperature in degrees Celsius
    * `yt`: nacelle temperature in degrees Celsius
    * `rbt`: rotor bearing temperature in degrees Celsius
* Mechanical – Power
    * `rm`: torque in Nm
    * `rs`: rotor speed in rpm
    * `p`: active power (i.e. "real power") in kW
    * `q`: reactive power in kVAr
* Weather
    * `temp`: the outside temperature in degrees Celsius
    * `pressure`: the outside pressure in hPa
    * `humidity`: the outside humidity in %
    * `wind_speed`: the wind speed in m/s
    * `wind_deg`: the wind direction in degrees
    * `rain_1h`: the quantity of rain in the last hour in mm
    * `snow_1h`: the quantity of snow in the last hour in mm

There is immediately some potential redundancy here: we have three wind speed measurements that, assuming `ws` is reliable, can be condensed to one, saving us two million floating point numbers (in space and future compute time). Similarly, we have the `wind_speed`, `wind_deg` and `temp` variables taken from the openweathermap.org source, which we shall assume are less reliable than the actual measurements taken on the ground.

For this reason, we will only read in 17 of the 21 available metrics.

In [6]:
df: pd.DataFrame = pd.read_sql("""
SELECT
    wind_turbine_name,
    datetime,
    ws, wa,
    ba, va, ya,
    ot, yt, rbt,
    rm, rs, p, q,
    temp, pressure, humidity,
    rain_1h, snow_1h
FROM
    wind_turbines.observations
""", conn)

In [7]:
df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
df = df.sort_values(['wind_turbine_name','datetime']).reset_index(drop=True)

Let's now check for nulls, nans and zeros in the data, for reasons discussed above.

In [8]:
def summarise_missing_data(df: pd.DataFrame) -> pd.DataFrame:
    nulls: pd.Series = df.isnull().sum()
    nans: pd.Series = df.isna().sum()
    zeros: pd.Series = (df == 0).sum()
    return pd.concat([nulls, nans, zeros], axis=1, keys=['nulls', 'nans', 'zeros'])

summarise_missing_data(df)

,nulls,nans,zeros
wind_turbine_name,0,0,0
datetime,0,0,0
ws,0,0,20060
wa,0,0,21
ba,0,0,2912
va,0,0,503
ya,0,0,129
ot,0,0,269
yt,0,0,11
rbt,0,0,61


Looks good! There are no nulls or nans, and while zeroes are present, this is to be expected to some degree (e.g. it is possible there is no wind, or the turbine is not operating and no power is being generated).

Having established the baseline quality of these numerical variables, the typical next step is to visualise them. This can be done with some simple histograms and time series plots.

In [9]:
metrics: list[str] = """
ws, wa,
ba, va, ya,
ot, yt, rbt,
rm, rs, p, q,
temp, pressure, humidity,
rain_1h, snow_1h
""".replace(',', '').split()

turbines: list[str] = ['R80711', 'R80721', 'R80736', 'R80790']
colors: list[str] = ['red', 'blue', 'green', 'orange']

In [10]:
# Plot histograms for each turbine and metric
fig, axes = plt.subplots(4, len(metrics), figsize=(100, 20))
for i, turbine in enumerate(turbines):
    for j, metric in enumerate(metrics):
        df[df['wind_turbine_name'] == turbine][metric].hist(
            ax=axes[i, j], color=colors[i], bins=50
        )
        axes[i, j].set_title(f"{turbine} - {metric}")
fig.suptitle('Histograms of metrics for each wind turbine', fontsize=20)
fig.tight_layout()
fig.savefig('./outputs/visuals/histograms.png')
plt.close()

In [11]:
# Plot time series for each turbine and metric
df['yearquarter'] = df['datetime'].dt.to_period('Q')
fig, axes = plt.subplots(4, len(metrics), figsize=(100, 20))
colors = ['red', 'blue', 'green', 'orange']
for i, turbine in enumerate(['R80711', 'R80721', 'R80736', 'R80790']):
    for j, metric in enumerate(metrics):
        # make sure to hide the xaxis labels
        df[df['wind_turbine_name'] == turbine].boxplot(
            metric, by='yearquarter', ax=axes[i, j], color=colors[i],
            grid=False, rot=90
        )
        axes[i, j].set_xlabel('')
        axes[i, j].set_title(f"{turbine} - {metric}")
fig.suptitle('Quarterly boxplots of metrics for each wind turbine', fontsize=20)
fig.tight_layout()
fig.savefig('./outputs/visuals/boxplots.png')
plt.close()

df.drop(columns=['yearquarter'], inplace=True)

/var/folders/3s/82yptz8s4hqf4zcllng8snvr0000gn/T/ipykernel_28534/3736903269.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['yearquarter'] = df['datetime'].dt.to_period('Q')


In [12]:
# Plot pair-wise scatter plots (for some metrics)
selected_metrics: list[str] = ['ws', 'ba', 'va', 'ot', 'yt', 'rbt', 'rm', 'rs', 'p', 'q'] + ['wind_turbine_name']
df_sampled = df.groupby('wind_turbine_name').apply(lambda x: x.sample(1000))
sns.pairplot(
    df_sampled[selected_metrics],
    hue='wind_turbine_name',
    corner=True
)
plt.savefig('./outputs/visuals/pairwise_scatter.png')
plt.close()

For fun, let's also look at a wind rose plot across each of the four turbines.

In [13]:
for turbine in turbines:
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='windrose')
    ax.bar(
        df[df['wind_turbine_name'] == turbine]['wa'],
        df[df['wind_turbine_name'] == turbine]['ws'],
        normed=True, opening=0.8, edgecolor='white'
    )
    ax.set_title(f'{turbine} - Windrose')
    fig.savefig(f'./outputs/visuals/windrose_{turbine}.png')
    plt.close()

Studying these visuals, we can observe several important idiosyncracies in the data:

* Univariately...
    * Observed wind speed and angle are very similar across the four turbines. This is to be expected, given their physical proximity. They are also relatively consistent across time, suggesting that the area in which the turbines are situated is subject to relatively stable weather conditions.
    * Pitch angle looks to be mostly set around one of three values (0deg, 45deg, and 90deg), though there are some strange positive and negative outliers for two of the turbines.
    * Vane angle is symmetrically distributed about zero, with a very similar distribution across the four turbines. This is to be expected, given the wind vane is located on the nacelle, and hence will typically be in line with the wind direction.
    * Nacelle angle is again consistent across the four turbines, and largely aligns with wind direction.
    * The distribution of oil temperatures, nacelle temperatures and rotor bearing temperatures looks similar across the four turbines, though there are some large positive and negative outliers for two of the turbines. They also track expected seasonal patterns, with higher temperatures in the summer and lower temperatures in the winter.
    * Rotor torque shows a large spike around zero, and then a more normal distribution around a positive mean. This is likely due to the turbine being switched off.
    * Rotor speed is meanwhile either very low (less than 2 rpm) or high (greater than 9 rpm), with another large spike at 17 rpm. This conforms with the cut-in and  cut-out speeds of the turbine.
    * Both rotor speed and torque do not exhibit any obvious changes over time.
    * Active power is mostly zero, with a long right tail
    * Reactive power is distributed bimodally, with some large negative outliers
    * The weather metrics (temperature, pressure, humidity, rain and snow) are by construction distributed identically across the four turbines. They also show the expected seasonal patterns, with higher temperatures, lower pressures and lower humidity in the summer, and vice versa in the winter.
* Bivariately...
    * As wind speed increases, vane angle tends to focus around zero. This is to be expected, as the wind vane is located on the nacelle, and hence will typically be in line with the wind direction.
    * There are no obvious correlations between wind speed and mechanical temperatures.
    * There is a sigmoidal relationship between wind speed and rotor torque. Meanwhile, torque is almost linearly related to active power output, meaning wind speed and power are sigmoidally related, as we would expect.
    * Rotor speed and active power output are positively related, though as we have seen, rotor speed maxxes out, and power output can continue to increase beyond this limit.
    * While we do not expect a priori any relationship between active and reactive power, the latter correlates positively with wind speed, rotor torque, rotor speed and active power. Not too sure why!


There is much to unpack, but we can start by addressing some of the 'outlier' values.

## Data Cleaning

Now that we know a little about wind turbines and have explored the raw data, we are ready to clean the data. Given what we know about the physical processes concerned, and what we have seen in the shape of the data, I claim a parametric truncation approach is the most suitable for this dataset. This means we will assume the data are distributed according to some parametric distribution (e.g. Gaussian, Poisson, etc.) and then truncate the data at some threshold (e.g. 3 standard deviations from the mean).

My rationale for this approach is... we are investigating physical processes which are in a very real sense bounded, so negative one hundred degrees celcius does not make sense, nor does a wind speed of 1000 m/s. Physical systems abide by physical laws, and so we can expect the data to be distributed according to some parametric distribution. This is not always the case, but it is a reasonable assumption to make in the absence of domain expertise. "It is more likely that the weird outliers are errors than they are truly extreme values" is the basic idea, where our definition of "extreme" is based on the observed distributions that most of the data follow.

**In fact, not so much cleaning was required! Here is all I did:**

* ~~Pitch angle (`ba`) surely cannot be negative...~~
* Oil temperature (`ot`), nacelle temperature (`yt`), and rotor bearing temperature (`rbt`) need to be truncated for some weird negative values.
* ~~Reactive power is not very consistent arnd there are some weird negative values. Surely these are errors...~~
* The external weather variables (pressure and humidity) are in fact integer-valued; to smooth them out, we can convolve them with a Gaussian kernel.

In [14]:
# Drop any values of ot that are below -10...
df = df[df['ot'] > -10]
# ...and any values of yt that are below 0...
df = df[df['yt'] > 0]
# ...and any values of rbt that are below 0...
df = df[df['rbt'] > 0]

In [15]:
# Add some random noise to the pressure and humidity variables...
df['pressure'] = df['pressure'] + np.random.normal(0, 0.5, len(df))
df['humidity'] = df['humidity'] + np.random.normal(0, 0.5, len(df))
# ...making sure to clip the humidity values to be between 0 and 100
df['humidity'] = df['humidity'].clip(0, 100)

## Feature Engineering

Now that we have clean data, we can consider engineering any new features that might be useful for our models. This is a very open-ended process and would signficantly benefit from domain expertise... but for the moment we just have me and my own ideas:

* `ws_sq`: wind speed squared, since rotor torque is proportional to the square of wind speed
* `ws_cb`: wind speed cubed, since rotor power is proportional to the cube of wind speed
* `ws_a`: actual wind speed, since the wind speed measured by the anemometer is not the same as that experienced by the turbine if it is not facing directly into the wind
* `temp_6hr`: a rolling average of temperature over the past 6 hours, to capture the persistent effect of temperature on the turbine
* `rho`: pressure divided by temperature, as a proxy for air density
* `rs_n`: normalised rotor speed, as a ratio to maximum rotor speed (17 rpm)
* `ro`: whether the rotor is running without wind power, i.e. wind speed is zero but wind speed is not
* `ts`: tip speed ratio, as a ratio of rotor speed to wind speed
* `rsm`: rotor speed-torque gradient
* `o`: a boolean, indicating whether the wind speed is within the operational range of the turbine (i.e. between cut-in and cut-out speeds)
* `tg`: the ratio of temperatures outside and inside the nacelle, as a proxy for the efficiency of the cooling system
* `s`: apparent power, defined as $S = \sqrt{P^2 + Q^2}$
* `pf`: the power factor, defined as $\cos \phi = \frac{P}{S}$
* Four indicators for each six-hour block of the day, to capture varying grid demand:
    * `d1`: 00:00 - 06:00
    * `d2`: 06:00 - 12:00
    * `d3`: 12:00 - 18:00
    * `d4`: 18:00 - 00:00

In [16]:
df['ws_sq'] = df['ws'] ** 2
df['ws_cb'] = df['ws'] ** 3
df['ws_a'] = np.cos(df['va']) * df['ws']

df['temp_6h'] = (
    df
    .groupby('wind_turbine_name')['temp']
    .rolling(6).mean()
    .reset_index(0, drop=True)
)

df['rho'] = df['pressure'] / (df['temp'] + 273.15)

df['rs_n'] = df['rs'] / 17.0
df['ro'] = np.where((df['ws'] == 0) & (df['rs'] > 0), 1, 0)
df['ts'] = df['rs'] / df['ws']
df['rsm'] = df['rs'] / df['rm']

df['o'] = df['ws'].map(lambda x: 1 if x > 3.5 and x < 25 else 0)

df['tg'] = df['yt'] / df['ot']

df['s'] = np.sqrt(df['p'] ** 2 + df['q'] ** 2)
df['pf'] = df['p'] / df['s']

df['d1'] = np.where((df['datetime'].dt.hour >= 0) & (df['datetime'].dt.hour < 6), 1, 0)
df['d2'] = np.where((df['datetime'].dt.hour >= 6) & (df['datetime'].dt.hour < 12), 1, 0)
df['d3'] = np.where((df['datetime'].dt.hour >= 12) & (df['datetime'].dt.hour < 18), 1, 0)
df['d4'] = np.where((df['datetime'].dt.hour >= 18) & (df['datetime'].dt.hour < 24), 1, 0)

In [17]:
df = df[~df['temp_6h'].isnull()]

df['ts'].fillna(0, inplace=True)
df['rsm'].fillna(0, inplace=True)
df['tg'].fillna(0, inplace=True)

## Summary

In this notebook, we have learned about wind turbines, explored the raw data, cleaned the data, and engineered some new features. We are now ready to start modelling!

In [18]:
df.to_sql(
    name='observations_clean',
    con=engine,
    schema='wind_turbines',
    if_exists='replace',
    index=False
)

500

In [19]:
conn.close()

### References

#### Ideas

* Craft better features to track grid demand patterns!

#### Questions

* Not really sure what the vane angle is. Is it the angle of the wind vane? Perhaps this is why it is symmetrically distributed about zero... because the wind vane is located on the nacelle, and hence will typically be in line with the wind direction.

#### Assumptions

* It is reasonable to just use `ws` and drop `ws_1`, `ws_2`, and `wind_speed`. Similarly, `wa` is strictly more accurate than `wind_deg`, so the latter variables can be dropped without any consequence. We could test this properly, using something like Kolmogorov-Smirnov... or maybe they are actually tracking different physical measurements?
* The actual weather measurements taken by the wind turbines are more reliable than those provided by the third-party weather data provider.
* It is legitimate to just ignore the 'outliers' and focus on the 'normal' data. This is a common approach in machine learning, but may not be appropriate here. It is possible that the outliers are actually the most interesting data points, and that the 'normal' data is actually the most boring. This is a question for the domain experts.
* Data truncation is the most appropriate form of data cleaning... and all of my assumptions about certain physical values being impossible are valid.